# Técnicas de resample 

Neste notebook, transformarei os dados utilizando técnicas de resample para podermos treinar os modelos

Mateus de Sousa Martins

mateus.martins8605@gmail.com

In [1]:
#importando bibliotecas 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
credit = pd.read_csv('creditcard.csv') #importando o arquivo csv obtidono site da Kaggle para mais informações https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
credit.head() #visualizando as 5 primeiras linhas

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Conteúdo do DataFrame
* Time indica o tempo, em segundos, que se passou desde a primeira transação registrada;
* V1 a V28 passaram por uma transformação PCA para proteger os dados dos clientes, não é possível afirmar o que cada coluna significa;
* Amount significa o valor da transação;
* Class indica se houve ou não fraude, 0 se não houve, 1 caso contrário.

In [3]:
print(f'Temos {credit.shape[0]} amostra de dados.')
print(f'Existem {credit.shape[1]-1} variáveis') #-1 para excluir a coluna class, que não será uma variável e sim o alvo do modelo

Temos 284807 amostra de dados.
Existem 30 variáveis


In [4]:
# Porcentagem de dados que indicam fraude
print(f'A porcentagem de dados que indicam a ocorrência de fraude é de {credit["Class"].mean()*100:.2f}%')

A porcentagem de dados que indicam a ocorrência de fraude é de 0.17%


In [5]:
credit.Class.value_counts() # Existem 492 ocorrências de fraude

0    284315
1       492
Name: Class, dtype: int64

In [6]:
#Mudando a variável amount por uma discreta
disc = np.linspace(credit['Amount'].min(),credit['Amount'].max(),10)
disc[-1] = np.inf
disc[0] = -np.inf
credit['Amount'] = pd.cut(credit['Amount'],bins= disc,labels=list(range(1,len(disc))))

In [7]:
credit.Amount.value_counts()

1    284484
2       285
3        28
4         4
5         3
7         2
9         1
6         0
8         0
Name: Amount, dtype: int64

In [8]:
credit_mod = credit[credit['Class']==1]
credit_mod.Amount.value_counts()
# Todos os golpes ocorrem para pequenos valores, entre 0 e 2854

1    492
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
Name: Amount, dtype: int64

In [11]:
X = credit.iloc[:,:-1]
y = credit.iloc[:,-1]

In [13]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X,y, test_size=0.2)

# Resample

A biblioteca imblearn é utilizada para tratamento de dados em aprendizado desbalanceado. Ela funciona bem com as bibliotecas sklearn e tensorflow.

In [14]:
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek

Tomek Links toma a distância entre um dado da classe minoritária e o seu par mais próxima da classe majoritária. Existe uma distância de threshold de modo que se a distância entre esses pares for menos, o dado da classe majoritária é eliminado, por padrão, também é possível modificar para apagar os dados da classe minoritária ou o par.

SMOTE é o nome da técnica que cria dados artificiais da classe minoritária introduzindo ruído aos dados. SOMOTETomek aplica SMOTE e depois Tomek.

In [15]:
# Começando por TomekLinks
tl = TomekLinks(sampling_strategy = 'majority')

X_tl, y_tl = tl.fit_resample(X,y)

print(f"Quantidade de Links removidos {X.shape[0]-X_tl.shape[0]}")


Quantidade de Links removidos 17


In [16]:
st = SMOTETomek(sampling_strategy = 'auto')
X_st, y_st = st.fit_resample(X,y)

# Exportando os dados

In [18]:
df_tl = pd.concat([X_tl,y_tl],axis=1)
df_st = pd.concat([X_st,y_st],axis=1)

In [19]:
df_tl.to_csv('credit_tl.csv')
df_st.to_csv('credit_st.csv')

In [22]:
df_teste = pd.concat([X_test,y_test],axis=1)
df_teste.to_csv('credit_teste.csv')